### Mobility processing
This notebook works through the process of adapting Google mobility data into the required format for the model.

In [ ]:
import pandas as pd

from emutools.tex import StandardTexDoc
from emutools.calibration import round_sigfig
from aust_covid.inputs import get_raw_state_mobility
from aust_covid.utils import add_image_to_doc
from aust_covid.mobility import get_non_wa_mob_averages, get_relative_mobility, map_mobility_locations
from aust_covid.plotting import plot_state_mobility, plot_processed_mobility
from inputs.constants import MOBILITY_MAP, SUPPLEMENT_PATH, MOBILITY_AVERAGE_WINDOW

In [ ]:
app_doc = StandardTexDoc(SUPPLEMENT_PATH, 'mobility', 'Mobility processing', 'austcovid')

In [ ]:
state_data, jurisdictions, mob_locs = get_raw_state_mobility(app_doc)
fig = plot_state_mobility(state_data, jurisdictions, mob_locs)
add_image_to_doc(fig, 'state_mobility', 'Raw state-level mobility obtained from Google.', app_doc, 'Mobility')
fig

In [ ]:
mapping_table = pd.DataFrame(MOBILITY_MAP)
mapping_table.index = mapping_table.index.str.replace('_', ' ').str.capitalize()
mapping_table.columns = mapping_table.columns.str.replace('_', ' ').str.capitalize()
mapping_table = mapping_table.applymap(str)
app_doc.include_table(mapping_table, 'Mobility')
mapping_table

In [ ]:
wa_data = state_data.loc[state_data['sub_region_1'] == 'Western Australia', mob_locs]
state_averages = get_non_wa_mob_averages(state_data, mob_locs, jurisdictions, app_doc)
non_wa_relmob = get_relative_mobility(state_averages)
wa_relmob = get_relative_mobility(wa_data)
processed_mob = map_mobility_locations(wa_relmob, non_wa_relmob, app_doc)
smoothed_mob = processed_mob.rolling(MOBILITY_AVERAGE_WINDOW).mean().dropna()
squared_mob = smoothed_mob ** 2.0

In [ ]:
mobility_types = {
    'original': processed_mob,
    'smoothed': smoothed_mob,
    'smoothed squared': squared_mob,
}
fig = plot_processed_mobility(mobility_types)
add_image_to_doc(fig, 'processed_mobility', 'Processed mobility for model.', app_doc, 'Mobility')
fig

In [ ]:
app_doc.write_doc()